# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [31]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy import stats #The SciPy stats module
import yfinance as yf
import tqdm.notebook as tqdm

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [24]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks.Ticker=stocks.Ticker.apply(lambda tck: tck.replace(".","-"))

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [22]:
symbol = 'AAPL'
data = yf.Ticker(symbol)

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [23]:
data.history(period="1y")['Close'].resample('Y').ffill().pct_change()[-1]

0.8659351593134377

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [25]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(' '.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [33]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in tqdm.tqdm(symbol_strings):
    data = yf.Tickers(symbol_string).tickers
    errors=[]
    for t in tqdm.tqdm(data):
        try:
            final_dataframe = final_dataframe.append(
                                        pd.Series([t.ticker, 
                                                   t.info['previousClose'],
                                                   t.history(period="1y")['Close'].resample('Y').ffill().pct_change()[-1],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        except:
            errors.append(t.ticker)
        
    
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,117.31,0.385050,N/A
1,AAL,15.66,-0.433990,N/A
2,AAP,160.68,0.008870,N/A
3,AAPL,131.97,0.872460,N/A
4,ABBV,103.26,0.228193,N/A
...,...,...,...,...
497,YUM,107.54,0.107375,N/A
498,ZBH,149.25,0.007466,N/A
499,ZBRA,376.89,0.493815,N/A
500,ZION,43.34,-0.126609,N/A


In [34]:
final_dataframe.to_csv("year_return.csv",index=False)

## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [35]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:51]
final_dataframe.reset_index(drop = True, inplace = True)
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,LB,38.96,1.203472,N/A
1,NVDA,519.75,1.194435,N/A
2,PYPL,238.64,1.190718,N/A
3,ALB,149.64,1.102301,N/A
4,AMD,91.81,1.008921,N/A
5,CDNS,134.00,0.940456,N/A
6,NOW,553.89,0.925404,N/A
7,IDXX,497.14,0.894905,N/A
8,FCX,24.79,0.885015,N/A
9,ALGN,528.03,0.883888,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [36]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

portfolio_input()
print(portfolio_size)

1000000


In [37]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,LB,38.96,1.203472,503
1,NVDA,519.75,1.194435,37
2,PYPL,238.64,1.190718,82
3,ALB,149.64,1.102301,131
4,AMD,91.81,1.008921,213
5,CDNS,134.00,0.940456,146
6,NOW,553.89,0.925404,35
7,IDXX,497.14,0.894905,39
8,FCX,24.79,0.885015,790
9,ALGN,528.03,0.883888,37


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [44]:
hqm_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in tqdm.tqdm(symbol_strings):
    data = yf.Tickers(symbol_string).tickers
    errors=[]
    for t in tqdm.tqdm(data):
        try:
            hist=t.history(period="1y")['Close']
            hqm_dataframe = hqm_dataframe.append(
                                        pd.Series([t.ticker, 
                                                   t.info['previousClose'],
                                                   'N/A',
                                                   hist.resample('Y').ffill().pct_change()[-1],
                                                   'N/A',
                                                   hist.resample('6M').ffill().pct_change()[-1],
                                                   'N/A',
                                                   hist.resample('3M').ffill().pct_change()[-1],
                                                   'N/A',
                                                   hist.resample('M').ffill().pct_change()[-1],
                                                   'N/A',
                                                   'N/A'
                                                   ], 
                                                  index = hqm_columns), 
                                        ignore_index = True)
        except:
            errors.append(t.ticker)
        
hqm_dataframe.columns

Index(['Ticker', 'Price', 'Number of Shares to Buy', 'One-Year Price Return',
       'One-Year Return Percentile', 'Six-Month Price Return',
       'Six-Month Return Percentile', 'Three-Month Price Return',
       'Three-Month Return Percentile', 'One-Month Price Return',
       'One-Month Return Percentile', 'HQM Score'],
      dtype='object')

## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [45]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row, f'{time_period} Price Return'])/100

# Print each percentile score to make sure it was calculated properly
for time_period in time_periods:
    print(hqm_dataframe[f'{time_period} Return Percentile'])

#Print the entire DataFrame    
hqm_dataframe

0       0.850598
1      0.0239044
2       0.386454
3       0.976096
4       0.699203
         ...    
497     0.547809
498     0.384462
499     0.918327
500     0.203187
501     0.695219
Name: One-Year Return Percentile, Length: 502, dtype: object
0      0.681275
1      0.494024
2      0.290837
3       0.89243
4      0.197211
         ...   
497    0.591633
498    0.537849
499    0.888446
500    0.631474
501    0.388446
Name: Six-Month Return Percentile, Length: 502, dtype: object
0       0.505976
1       0.784861
2         0.2251
3       0.535857
4       0.587649
         ...    
497     0.613546
498     0.364542
499     0.948207
500     0.942231
501    0.0956175
Name: Three-Month Return Percentile, Length: 502, dtype: object
0      0.328685
1      0.934263
2      0.814741
3      0.948207
4      0.213147
         ...   
497    0.573705
498    0.316733
499    0.386454
500    0.922311
501    0.324701
Name: One-Month Return Percentile, Length: 502, dtype: object


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,117.31,N/A,0.384224,0.850598,0.330107,0.681275,0.164469,0.505976,0.003678,0.328685,N/A
1,AAL,15.66,N/A,-0.438133,0.0239044,0.228386,0.494024,0.306347,0.784861,0.136235,0.934263,N/A
2,AAP,160.68,N/A,0.004962,0.386454,0.122822,0.290837,0.040309,0.2251,0.081161,0.814741,N/A
3,AAPL,131.97,N/A,0.870124,0.976096,0.497760,0.89243,0.177358,0.535857,0.143343,0.948207,N/A
4,ABBV,103.26,N/A,0.230577,0.699203,0.078867,0.197211,0.194725,0.587649,-0.012813,0.213147,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
497,YUM,107.54,N/A,0.108791,0.547809,0.272708,0.591633,0.205616,0.613546,0.035728,0.573705,N/A
498,ZBH,149.25,N/A,0.004503,0.384462,0.253931,0.537849,0.097441,0.364542,0.001916,0.316733,N/A
499,ZBRA,376.89,N/A,0.495889,0.918327,0.492909,0.888446,0.513547,0.948207,0.009751,0.386454,N/A
500,ZION,43.34,N/A,-0.128408,0.203187,0.306161,0.631474,0.505230,0.942231,0.129956,0.922311,N/A


In [46]:
hqm_dataframe.to_csv("high_quality_momentum_data.csv")

## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [47]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [48]:
hqm_dataframe.sort_values(by = 'HQM Score', ascending = False)
hqm_dataframe = hqm_dataframe[:51]

## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [49]:
portfolio_input()

In [50]:
position_size = float(portfolio_size) / len(hqm_dataframe.index)
for i in range(0, len(hqm_dataframe['Ticker'])-1):
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe['Price'][i])
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,117.31,167,0.384224,0.850598,0.330107,0.681275,0.164469,0.505976,0.003678,0.328685,0.591633
1,AAL,15.66,1252,-0.438133,0.0239044,0.228386,0.494024,0.306347,0.784861,0.136235,0.934263,0.559263
2,AAP,160.68,122,0.004962,0.386454,0.122822,0.290837,0.040309,0.2251,0.081161,0.814741,0.429283
3,AAPL,131.97,148,0.870124,0.976096,0.497760,0.89243,0.177358,0.535857,0.143343,0.948207,0.838147
4,ABBV,103.26,189,0.230577,0.699203,0.078867,0.197211,0.194725,0.587649,-0.012813,0.213147,0.424303
5,ABC,96.97,202,0.157701,0.621514,-0.032387,0.0677291,0.001937,0.139442,-0.062118,0.0278884,0.214143
6,ABMD,303.41,64,0.815288,0.97012,0.281959,0.603586,0.117700,0.416335,0.129770,0.920319,0.72759
7,ABT,108.35,180,0.257077,0.74502,0.184206,0.408367,-0.008975,0.12749,-0.006699,0.241036,0.380478
8,ACN,257.30,76,0.253697,0.737052,0.219230,0.474104,0.154199,0.486056,0.043077,0.619522,0.579183
9,ADBE,499.86,39,0.509991,0.922311,0.144035,0.336653,0.015456,0.169323,0.040839,0.609562,0.509462


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [51]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [52]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [53]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', integer_template]
                    }

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [54]:
writer.save()